# Phase 3: Microservice Identification (Grouping by Similar Services)

In [ ]:
version = "v_imen" # All options: v_imen, v_team
system = "pos" # All options: jforum, cargotracker, petclinic, pos
model_type = "codebert" # All options: ft_codebert, word2vec, albert, codebert, roberta, bert
best_community_detection_algorithm = 'Louvain' # All options: Louvain, EdMot, Infomap, LabelPropagation, FastGreedy, GirvanNewman

## 1.1 Create service graph

In [ ]:
import pandas as pd
import numpy as np
from scipy import spatial

In [ ]:
# Load the data
communities_df = pd.read_csv(f"generated_data/community/{version}_{system}_{best_community_detection_algorithm}_communities.csv")
class_graph_df = pd.read_csv(f"generated_data/graph/class/{version}_{system}_class_graph.csv")
embeddings_df = pd.read_csv(f"generated_data/embedding/{version}_{system}_{model_type}_embeddings.csv")

# Extract class names and their embeddings
class_names = embeddings_df.iloc[:, 0].str.split(';', expand=True)[0]
embeddings = embeddings_df.iloc[:, 1:].values
class_embeddings_dict = dict(zip(class_names, embeddings))

def compute_static_distances_for_service_pairs(class_graph, communities):
    merged_df = class_graph.merge(communities, left_on='class1', right_on='class_name').merge(
        communities, left_on='class2', right_on='class_name', suffixes=('_1', '_2')
    )
    inter_service_df = merged_df[merged_df['service_1'] != merged_df['service_2']]
    return inter_service_df.groupby(['service_1', 'service_2'])['static_distance'].sum().to_dict()

def compute_service_embeddings(embeddings_dict, communities):
    """Compute service embeddings by averaging class embeddings for each service, skipping missing embeddings."""
    service_embeddings = {}
    for service, class_group in communities.groupby('service')['class_name']:
        class_embeddings = [embeddings_dict[class_name] for class_name in class_group if class_name in embeddings_dict]
        if class_embeddings:
            service_embeddings[service] = np.mean(class_embeddings, axis=0)
    return service_embeddings

def compute_semantic_distances_for_service_pairs(embeddings_dict, communities):
    service_embeddings = compute_service_embeddings(embeddings_dict, communities)
    
    services = list(service_embeddings.keys())
    semantic_distances = {}
    for i, s1 in enumerate(services):
        for j, s2 in enumerate(services):
            if i != j:
                distance = 1 - spatial.distance.cosine(service_embeddings[s1], service_embeddings[s2])
                semantic_distances[(s1, s2)] = distance
    
    return semantic_distances

def normalize_data(data):
    min_val, max_val = min(data.values()), max(data.values())
    range_val = max_val - min_val
    return {k: (v - min_val) / range_val for k, v in data.items()} if range_val else {k: 0 for k, v in data.items()}

# Compute static and semantic distances
static_distances = compute_static_distances_for_service_pairs(class_graph_df, communities_df)
semantic_distances = compute_semantic_distances_for_service_pairs(class_embeddings_dict, communities_df)

# Normalize the distances
normalized_static_distances = normalize_data(static_distances)
normalized_semantic_distances = normalize_data(semantic_distances)

# Create the service graph DataFrame
service_graph_data = [
    [s1, s2, normalized_static_distances.get((s1, s2), 0), normalized_semantic_distances.get((s1, s2), 0)]
    for s1, s2 in static_distances.keys()
]
service_graph_df = pd.DataFrame(service_graph_data, columns=['service1', 'service2', 'static_distance', 'semantic_distance'])

# Save the DataFrame
filename = f"generated_data/graph/service/{version}_{system}_service_graph.csv"
service_graph_df.to_csv(filename, index=False)

## 1.2 Cluster services

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
from sklearn.manifold import MDS
import skfuzzy as fuzz
from utils import save_microservices_to_file, save_clusters_to_file

In [ ]:
def compute_edge_weight(semantic, static, alpha=0.5):
    if not (0 <= semantic <= 1) or not (0 <= static <= 1):
        raise ValueError("Both 'semantic' and 'static' values should be between 0 and 1.")
    
    beta = 1 - alpha
    return alpha * static + beta * semantic

In [ ]:
services_graph = nx.DiGraph([
    (row['service1'], row['service2'], {"weight": compute_edge_weight(row['semantic_distance'], row['static_distance'])})
    for _, row in service_graph_df.iterrows()
])

# Remove all edges with a weight of 0 to simplify the graph
services_graph.remove_edges_from([
    (u, v) for u, v, weight in services_graph.edges(data='weight') if weight == 0
])

# Print nodes and edge weights
for u, v, weight in services_graph.edges(data='weight'):
    print(f"{u} -> {v}: {weight}")

# Generate a list of nodes in the graph
nodes_list = list(services_graph.nodes)

# Initialize a matrix to hold pairwise distances with a default value
# indicating maximum dissimilarity
default_distance = 1
distances = {
    node: {node2: default_distance for node2 in nodes_list}
    for node in nodes_list
}

# Update the distance matrix with the inverse of the edge weights
for u, v, data in services_graph.edges(data=True):
    distances[u][v] = distances[v][u] = 1 - data['weight']

# Convert the distances dictionary to a DataFrame and then to a matrix
dissimilarity_matrix = pd.DataFrame(distances).values

# Set the diagonal of the distance matrix to 0 (distance to self is 0)
np.fill_diagonal(dissimilarity_matrix, 0)

# Output the dissimilarity matrix
print("Dissimilarity matrix:")
print(dissimilarity_matrix)

### 1.2.1 Fuzzy C-means from Scikit

In [ ]:
# Constants
FUZZINESS = 2
ERROR_THRESHOLD = 0.005
MAX_ITERATIONS = 1000
CLUSTER_RANGE = range(1, 100)
RELATIVE_THRESHOLD = 1  # Membership threshold: adjust as needed

In [ ]:
def determine_optimal_clusters(data):
    """Determines the optimal number of clusters using the Elbow method."""
    fpc_values = []
    for c_value in CLUSTER_RANGE:
        _, _, _, _, _, _, fpc = fuzz.cmeans(
            data.T, 
            c=c_value, 
            m=FUZZINESS, 
            error=ERROR_THRESHOLD, 
            maxiter=MAX_ITERATIONS
        )
        fpc_values.append(fpc)

    plt.figure()
    plt.plot(CLUSTER_RANGE, fpc_values)
    plt.title('Fuzzy Partition Coefficient (FPC) for different cluster numbers')
    plt.xlabel('Number of clusters')
    plt.ylabel('FPC')
    plt.grid(True)
    plt.show()

    return detect_elbow(fpc_values)

def detect_elbow(y_values):
    """Detects the 'elbow' in a list of y-values."""
    # Get coordinates of all the points
    n_points = len(y_values)
    all_coords = np.vstack((range(n_points), y_values)).T
    # Get vectors between all points from the first point to the last point
    first_point = all_coords[0]
    line_vector = all_coords[-1] - all_coords[0]
    line_vector_norm = line_vector / np.sqrt(np.sum(line_vector**2))
    
    # Get orthogonal vectors from the first point to all points
    vec_from_first = all_coords - first_point
    scalar_prod = np.sum(vec_from_first * line_vector_norm, axis=1)
    vec_from_first_parallel = np.outer(scalar_prod, line_vector_norm)
    vec_to_line = vec_from_first - vec_from_first_parallel
    
    # Compute the distance to the line
    dist_to_line = np.sqrt(np.sum(vec_to_line ** 2, axis=1))
    
    # Return the index of the point with max distance to the line
    idx_elbow = np.argmax(dist_to_line)
    return idx_elbow

def fuzzy_cmeans_clustering(data, node_labels, optimal_clusters, relative_threshold=RELATIVE_THRESHOLD):
    """Clusters the data using Fuzzy C-Means, returning only significant memberships."""
    cntr, u, _, _, _, _, _ = fuzz.cmeans(
        data.T,
        c=optimal_clusters,
        m=FUZZINESS,
        error=ERROR_THRESHOLD,
        maxiter=MAX_ITERATIONS
    )
    # Create a dictionary to store memberships for each node
    fuzzy_memberships = {label: [] for label in node_labels}
    
    # Normalize memberships for each node so that they sum to 1
    for i, label in enumerate(node_labels):
        membership_values = u[:, i]
        total_membership = np.sum(membership_values)
        normalized_memberships = membership_values / total_membership if total_membership else membership_values
        
        # Find the maximum normalized membership value for the node
        highest_membership = np.max(normalized_memberships)
        
        # Include memberships that are above the threshold relative to the highest membership
        for j in range(optimal_clusters):
            if normalized_memberships[j] / highest_membership >= relative_threshold:
                cluster_name = f'cluster{j+1}'
                fuzzy_memberships[label].append((cluster_name, normalized_memberships[j]))

    return fuzzy_memberships

In [ ]:
# Example usage
optimal_clusters = determine_optimal_clusters(dissimilarity_matrix)
print(f"Optimal number of clusters: {optimal_clusters}")
fuzzy_clusters = fuzzy_cmeans_clustering(dissimilarity_matrix, nodes_list, optimal_clusters)

# Print the fuzzy_clusters in a readable format
for node, memberships in fuzzy_clusters.items():
    membership_str = ', '.join(f"{cluster}: {membership:.2f}" for cluster, membership in memberships)
    print(f"{node}: [{membership_str}]")

# Save results
# You will need to modify the save_microservices_to_file function to handle the new format
# save_microservices_to_file(fuzzy_clusters, services_graph, communities_df, f"results/{version}_{system}_microservices.txt")

### 1.2.1 Custom fuzzy C-means

In [ ]:
class FuzzyCMeans:
    def __init__(self, m, membership_threshold, merge_threshold):
        """
        Initialize the FuzzyCMeans object with the specified parameters.
        
        Parameters:
        m (float): The fuzziness coefficient that determines the level of cluster fuzziness.
        membership_threshold (float): The proportion of the highest membership value 
                                      used as a cutoff for assigning services to additional clusters.
        merge_threshold (float): The threshold for determining when to merge two clusters
                                 based on the overlap of their members.
        """
        self.m = m
        self.membership_threshold = membership_threshold
        self.merge_threshold = merge_threshold
        self.coef = 2 / (m - 1)  # Precomputed coefficient for membership calculation.

    def calculate_membership(self, service_idx, center_idx, distances):
        """
        Calculate the fuzzy membership of a service to a cluster center.
        
        Parameters:
        service_idx (int): Index of the service for which membership is being calculated.
        center_idx (int): Index of the cluster center to which membership is being calculated.
        distances (list): Matrix of distances between services and cluster centers.
        
        Returns:
        float: The membership value of the service to the cluster center.
        """
        epsilon = 1e-10  # Small value to avoid division by zero.
        distance_to_center = distances[service_idx][center_idx]

        membership_sum = 0.0

        if distance_to_center == 0:
            return 1.0
        
        for other_center_idx in self.center_type_indices.values():
            if other_center_idx != service_idx:
                # Direct distances to other centers
                other_distance_to_center = distances[service_idx][other_center_idx]

                membership_ratio = distance_to_center / (other_distance_to_center + epsilon)
                membership_sum += membership_ratio ** self.coef

        # The membership value is the inverse of the sum, normalized by the number of centers
        return 1 / membership_sum if membership_sum > 0 else 0

        
    def cluster_services(self, services_list, distances, service_type):
        """
        Cluster services based on their membership values to each cluster center.
        
        Parameters:
        services_list (list): List of all services to be clustered.
        distances (list): Matrix of distances between services and cluster centers.
        service_type (str): The type of service that defines the cluster centers.
        
        Returns:
        dict: A dictionary with cluster centers as keys and lists of services as values.
        """
        # Step 1: Initialize clusters and identify services that are centers based on the service type.
        self.center_type_indices = {
            service: idx for idx, service in enumerate(services_list) if service.startswith(service_type)
        }

        # Step 2: Calculate raw membership values for each service to each cluster center.
        raw_memberships = {
            service: [] for service in services_list
        }
        for idx, service in enumerate(services_list):
            for center_service, center_idx in self.center_type_indices.items():
                membership_value = self.calculate_membership(idx, center_idx, distances)
                raw_memberships[service].append((center_service, membership_value))

        # Step 3: Normalize membership values so that they sum to 1 for each service.
        normalized_memberships = {}
        for service, memberships in raw_memberships.items():
            total_membership = sum(membership for _, membership in memberships)
            normalized_memberships[service] = [
                (center_service, membership / total_membership if total_membership else 0)
                for center_service, membership in memberships
            ]

        # Print the normalized memberships for each service.
        # for service, memberships in normalized_memberships.items():
        #     print(f"{service}: {memberships}")

        # Step 4: Assign services to clusters based on normalized membership values.
        clusters = {center_service: [] for center_service in self.center_type_indices}
        for service, memberships in normalized_memberships.items():
            # Determine the highest normalized membership value.
            highest_membership_value = max(memberships, key=lambda x: x[1])[1]
            # Determine the primary cluster for this service.
            primary_cluster = max(memberships, key=lambda x: x[1])[0]
            clusters[primary_cluster].append(service)
            
            # Assign service to other clusters where the membership value is above the relative threshold.
            relative_threshold = highest_membership_value * self.membership_threshold
            for center_service, membership_value in memberships:
                if membership_value >= relative_threshold and center_service != primary_cluster:
                    clusters[center_service].append(service)

        # Step 5: Merge clusters if they have a significant overlap as defined by merge_threshold.
        clusters = self.merge_clusters_based_on_overlap(clusters)

        # Step 6: Remove duplicate services from each cluster to maintain unique membership.
        for center_service, members in clusters.items():
            clusters[center_service] = list(set(members))

        # Step 7: Output the clusters in correct format.
        # First, create a dictionary to map cluster labels to indices.
        cluster_indices = {cluster: idx for idx, cluster in enumerate(clusters.keys())}

        # Then, create a dictionary to store the cluster memberships for each node.
        fuzzy_memberships = {node: [] for node in services_list}

        # Finally, populate the node_memberships dictionary with normalized membership values.
        for node, memberships in normalized_memberships.items():
            for cluster, membership in memberships:
                    if cluster in clusters and node in clusters[cluster]:
                        fuzzy_memberships[node].append(('cluster' + str(cluster_indices[cluster]), membership))

        return fuzzy_memberships


    def calculate_overlap(self, cluster_a, cluster_b):
        """
        Calculate the overlap between two clusters.
        
        Parameters:
        cluster_a (list): The first cluster to compare.
        cluster_b (list): The second cluster to compare.
        
        Returns:
        float: The ratio of the intersection to the union of the two clusters.
        """
        intersection = set(cluster_a).intersection(cluster_b)
        union = set(cluster_a).union(cluster_b)
        return len(intersection) / len(union) if union else 0

    def merge_clusters_based_on_overlap(self, clusters):
        """
        Merge clusters based on the overlap between their services.
        
        Parameters:
        clusters (dict): Dictionary with cluster centers as keys and lists of services as values.
        
        Returns:
        dict: The updated dictionary of clusters after merging.
        """
        # Find all pairs of clusters that have an overlap above the merge threshold.
        clusters_to_merge = []
        keys = list(clusters.keys())
        for i, key_i in enumerate(keys):
            for j in range(i + 1, len(keys)):
                key_j = keys[j]
                if self.calculate_overlap(clusters[key_i], clusters[key_j]) > self.merge_threshold:
                    clusters_to_merge.append((key_i, key_j))

        # Merge the identified clusters.
        for key_i, key_j in clusters_to_merge:
            if key_i in clusters and key_j in clusters:
                clusters[key_i].extend(clusters[key_j])
                clusters[key_i] = list(set(clusters[key_i]))
                del clusters[key_j]

        return clusters

In [ ]:
# Constants (change if needed)
FUZZINESS = 2
RELATIVE_MEMBERSHIP_THRESHOLD = 0.7 # lower means clustering more loosely (services can be in multiple clusters)
MERGE_THRESHOLD = 0.5 # higher means less clusters will be merged
CENTER_TYPE = 'Entity Service'

In [ ]:
# Usage
fuzzy_c_means = FuzzyCMeans(m=FUZZINESS, membership_threshold=RELATIVE_MEMBERSHIP_THRESHOLD, merge_threshold=MERGE_THRESHOLD) # change thresholds

# Perform clustering with the best thresholds
fuzzy_clusters = fuzzy_c_means.cluster_services(nodes_list, dissimilarity_matrix, service_type=CENTER_TYPE) # change service type

# Output clusters
print(fuzzy_clusters)

# Save results
# save_clusters_to_file(clusters, communities_df, f"results/{version}_{system}_microservices.txt")

### 1.2.2 Hierarchical clustering

In [ ]:
from scipy.cluster.hierarchy import linkage, fcluster
from scipy.spatial.distance import squareform

def hierarchical_fuzzy_membership(dissimilarity_matrix, nodes, max_d, relative_threshold):
    """
    Perform hierarchical clustering and return fuzzy memberships.

    Parameters:
    - dissimilarity_matrix: The matrix of dissimilarity between nodes.
    - nodes: The list of node labels.
    - max_d: The threshold to cut the dendrogram to form clusters.
    - relative_threshold: The relative threshold for determining significant membership.

    Returns:
    - A dictionary with node labels as keys and a list of (cluster, membership) tuples as values.
    """
    # Perform the hierarchical clustering
    Z = linkage(squareform(dissimilarity_matrix), method='average')
    clusters = fcluster(Z, max_d, criterion='distance')

    # Calculate the centroids of each cluster
    centroids = {i: np.mean(dissimilarity_matrix[clusters == i], axis=0) for i in np.unique(clusters)}

    # Initialize the fuzzy memberships dictionary
    fuzzy_memberships = {}

    for node_idx, node in enumerate(nodes):
        # Calculate distances to centroids
        distances = {f'cluster{i}': np.linalg.norm(centroids[i] - dissimilarity_matrix[node_idx]) for i in centroids}
        
        # Invert the distances to get membership strengths (higher distance -> lower membership)
        inverted_memberships = {cluster_label: 1 / (distance + 1e-5) for cluster_label, distance in distances.items()}
        
        # Normalize the inverted memberships so they sum to 1 for each node
        total_inverted = sum(inverted_memberships.values())
        normalized_memberships = {cluster_label: membership / total_inverted for cluster_label, membership in inverted_memberships.items()}
        
        # Determine the maximum membership value for the node
        max_membership = max(normalized_memberships.values())
        
        # Include memberships that are above the relative threshold
        significant_memberships = {
            cluster_label: membership
            for cluster_label, membership in normalized_memberships.items()
            if membership / max_membership >= relative_threshold
        }
        
        # Add the significant memberships to the fuzzy_memberships dictionary for the node
        fuzzy_memberships[node] = list(significant_memberships.items())

    return fuzzy_memberships

fuzzy_memberships = hierarchical_fuzzy_membership(dissimilarity_matrix, nodes_list, max_d=0.55, relative_threshold=0.9)

# Print the fuzzy_memberships in a readable format
for node, memberships in fuzzy_memberships.items():
    membership_str = ', '.join(f"{cluster}: {membership:.2f}" for cluster, membership in memberships)
    print(f"{node}: [{membership_str}]")

## 1.3 Modularity-based Optimization

In [ ]:
import itertools
from networkx.algorithms.community.quality import modularity

def convert_fuzzy_to_hard_partition(fuzzy_clusters):
    hard_partition = {}
    for node, memberships in fuzzy_clusters.items():
        # Assign the node to the cluster with the highest membership degree
        highest_membership_cluster = max(memberships, key=lambda x: x[1])[0]
        hard_partition.setdefault(highest_membership_cluster, set()).add(node)
    return list(hard_partition.values())

def optimize_clustering_parameters(graph, dissimilarity_matrix, clustering_function, parameter_ranges):
    """
    Optimize clustering parameters based on modularity.
    
    Parameters:
    - graph: The graph on which the clustering is to be performed.
    - distance_matrix: The precomputed dissimilarity matrix for the graph.
    - clustering_function: The clustering function to be optimized.
    - parameter_ranges: A dictionary where keys are parameter names and values are lists of parameter values to try.
    
    Returns:
    - A tuple containing the optimal parameters and the corresponding modularity score.
    """
    best_modularity = -1
    best_parameters = None

    # Convert directed graph to undirected for modularity calculation
    if graph.is_directed():
        graph = graph.to_undirected()
    
    # Generate all combinations of parameters
    keys, values = zip(*parameter_ranges.items())
    for parameter_combination in itertools.product(*values):
        params = dict(zip(keys, parameter_combination))

        # Perform clustering with the given parameters
        fuzzy_clusters = clustering_function(dissimilarity_matrix, **params)

        # Convert fuzzy clusters to a hard partition
        communities = convert_fuzzy_to_hard_partition(fuzzy_clusters)

        # Calculate modularity
        current_modularity = modularity(graph, communities)

        # Update best parameters if current modularity is greater than the best found so far
        if current_modularity > best_modularity:
            best_modularity = current_modularity
            best_parameters = params

    return best_parameters, best_modularity

In [ ]:
fuzzy_cmeans_params = {
    'optimal_clusters': range(2, 100),
}
best_fuzzy_cmeans_params, best_fuzzy_cmeans_modularity = optimize_clustering_parameters(
    services_graph,
    dissimilarity_matrix,
    fuzzy_cmeans_clustering,
    fuzzy_cmeans_params
)

# Example usage for Custom fuzzy C-means
custom_fuzzy_cmeans_params = {
    'm': range(2, 5),
    'membership_threshold': np.linspace(0.1, 1.0, 20),
    'merge_threshold': np.linspace(0.1, 1.0, 20),
}
best_custom_fuzzy_cmeans_params, best_custom_fuzzy_cmeans_modularity = optimize_clustering_parameters(
    services_graph,
    dissimilarity_matrix,
    lambda d, **params: FuzzyCMeans(**params).cluster_services(nodes_list, d, CENTER_TYPE),
    custom_fuzzy_cmeans_params
)

# Example usage for Hierarchical clustering
hierarchical_params = {
    'max_d': np.linspace(0.0, 1.0, 20)
}
best_hierarchical_params, best_hierarchical_modularity = optimize_clustering_parameters(
    services_graph,
    dissimilarity_matrix,
    lambda d, **params: fcluster(linkage(squareform(d), method='average'), **params),
    hierarchical_params
)

# Print the best parameters and their corresponding modularity scores
print("Fuzzy C-means:")
print(f"Best parameters: {best_fuzzy_cmeans_params}")
print(f"Best modularity: {best_fuzzy_cmeans_modularity}")
print()
print("Custom fuzzy C-means:")
print(f"Best parameters: {best_custom_fuzzy_cmeans_params}")
print(f"Best modularity: {best_custom_fuzzy_cmeans_modularity}")
print()
print("Hierarchical clustering:")
print(f"Best parameters: {best_hierarchical_params}")
print(f"Best modularity: {best_hierarchical_modularity}")


## 1.4 Evaluation